In [1]:
import requests
import json
import re
from tqdm.notebook import tqdm

In [2]:
def preprocess(question: str):
    question = question.replace('&amp;', '&').replace('&#39;', '\'').replace('&quot;', '\"')
    return re.sub('[{}]+', '', question)

In [3]:
with open("../datasets/DBpedia/smarttask_dbpedia_test_questions.json") as f: 
    test = json.load(f)

In [4]:
test[0]

{'id': 'dbpedia_16015', 'question': 'How many ingredients are in the grain} ?'}

In [5]:
pred = list()

for q in tqdm(test):
    question = q['question']
    res = requests.get(f"https://webengineering.ins.hs-anhalt.de:41020/predict_demo?question={question}").json()
    res_dict = {
        "id": q['id'],
        "category": res['category'],
        "type": res['answer_type'],
        "confidence": res['confidence']
    }
    pred.append(res_dict)

  0%|          | 0/4381 [00:00<?, ?it/s]

In [6]:
assert len(test) == len(pred)

In [14]:
for p in pred:
    if p['confidence'][0] < 0.5 and p['category'] == 'resource' and len(p['type']) > 1:
        p['type'] = p['type'][1:]

In [17]:
with open("../evaluation/dbpedia/system_output_test_threshold.json", 'w') as f:
    json.dump(pred, f, ensure_ascii=False, indent=4)

In [15]:
pred[3]['type']

['dbo:Agent']

In [16]:
pred[3]['confidence']

[0.0634345664020444]